In [1]:
from jax import config
config.update('jax_enable_x64', True)
from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns
import jax.random as jr
import pandas as pd

from uncprop.utils.experiment import Experiment
from uncprop.models.elliptic_pde.experiment import PDEReplicate

base_dir = Path('/Users/andrewroberts/Desktop/git-repos/bip-surrogates-paper')

/Users/andrewroberts/Desktop/git-repos/bip-surrogates-paper/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
key = jr.key(43223)
key, key_experiment = jr.split(key)

experiment_name = 'pde_experiment'
experiment_settings = {
    'name': experiment_name,
    'base_out_dir': base_dir / 'out' / experiment_name,
    'num_reps': 1,
    'base_key': key_experiment,
    'Replicate': PDEReplicate,
    'write_to_file': True,
}

setup_kwargs = {
    'n_design': 50
}

def make_subdir_name(setup_kwargs, run_kwargs):
    n = setup_kwargs['n_design']
    return f'n_design_{n}'

pde_experiment = Experiment(subdir_name_fn=make_subdir_name, 
                            **experiment_settings)



Using existing base output directory: /Users/andrewroberts/Desktop/git-repos/bip-surrogates-paper/out/pde_experiment


In [ ]:
results, failed_reps = pde_experiment(setup_kwargs=setup_kwargs)

In [ ]:
rep = results[0]

samp = rep.samples['exact']
posterior = rep.posterior

In [ ]:
for i in range(posterior.dim):
    plt.plot(samp[:,i])

In [ ]:
key_idx, key_prior_samp, key = jr.split(key, 3)

idx = jr.choice(key_idx, samp.shape[0], (10_000,))
post_samp_df = pd.DataFrame(samp[idx], columns=posterior.prior.par_names)
prior_samp_df = pd.DataFrame(posterior.prior.sample(key_prior_samp, n=10_000), columns=posterior.prior.par_names)

post_samp_df['dataset'] = 'posterior'
prior_samp_df['dataset'] = 'prior'
samp_df = pd.concat([prior_samp_df, post_samp_df], ignore_index=True)

sns.pairplot(samp_df, hue='dataset')

In [ ]:
postsamp = samp[-50:,:]
param, log_field, pde_solution, observable = posterior.likelihood.forward_model.forward_with_intermediates(postsamp)
xgrid = posterior.likelihood.forward_model.pde_settings.xgrid

plt.plot(xgrid, pde_solution.T, color='gray')
plt.show()


### Test

In [ ]:





fit_pde_surrogate(key: PRNGKey,
                      posterior: Posterior,
                      n_design: int,
                      design_method: str,
                      gp_train_args: dict | None = None,
                      verbose: bool = True,
                      jitter: float = 0.0,
                      **fit_kwargs